In [1]:
!git clone https://github.com/athms/evaluating-deeplight-transfer.git

Cloning into 'evaluating-deeplight-transfer'...
remote: Enumerating objects: 526, done.
remote: Total 526 (delta 0), reused 0 (delta 0), pack-reused 526
Receiving objects: 100% (526/526), 12.07 MiB | 11.92 MiB/s, done.
Resolving deltas: 100% (304/304), done.


In [2]:
%cd evaluating-deeplight-transfer/

/content/evaluating-deeplight-transfer


In [3]:
!pip install poetry

     |████████████████████████████████| 175 kB 4.3 MB/s 
     |████████████████████████████████| 91 kB 8.0 MB/s 
     |████████████████████████████████| 8.8 MB 30.7 MB/s 
     |████████████████████████████████| 40 kB 4.8 MB/s 
     |████████████████████████████████| 425 kB 31.1 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 48 kB 2.4 MB/s 
     |████████████████████████████████| 58 kB 2.7 MB/s 
     |████████████████████████████████| 58 kB 1.2 MB/s 
     |████████████████████████████████| 4.0 MB 35.8 MB/s 
     |████████████████████████████████| 461 kB 30.2 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip'

In [4]:
# edit TOML file with TF as 1.15.3

In [5]:
%%writefile /content/evaluating-deeplight-transfer/pyproject.toml

[tool.poetry]
name = "evaluating-deeplight-transfer"
version = "0.1.0"
description = "Transfer learning with DeepLight"
authors = ["Armin W. Thomas <athms@stanford.edu>"]

[tool.poetry.dependencies]
python = ">=3.6.1,<4.0"
boto3 = "1.12.34"
botocore = "1.15.39"
einops = "0.3.2"
Keras = "2.2.4"
matplotlib = "3.3.4"
nilearn = "0.7.0"
numpy = "1.19.5"
tensorflow = "1.15.3"
tqdm = "4.60.0"
pandas = "1.1.5"
innvestigate = "1.0.8"
h5py = "2.10.0"

[tool.poetry.dev-dependencies]

[build-system]
requires = ["poetry-core>=1.0.0"]
build-backend = "poetry.core.masonry.api"

Overwriting /content/evaluating-deeplight-transfer/pyproject.toml


In [6]:
!poetry update

Creating virtualenv evaluating-deeplight-transfer-a1Qw1P2X-py3.7 in /root/.cache/pypoetry/virtualenvs
Updating dependencies
Resolving dependencies... (216.9s)

Writing lock file

Package operations: 59 installs, 0 updates, 0 removals

  • Installing numpy (1.19.5): Pending...
  • Installing six (1.16.0): Pending...
  • Installing typing-extensions (4.1.1): Pending...
  • Installing zipp (3.6.0): Pending...
  • Installing typing-extensions (4.1.1): Pending...
  • Installing zipp (3.6.0): Pending...
  • Installing six (1.16.0): Downloading... 0%
  • Installing typing-extensions (4.1.1): Pending...
  • Installing zipp (3.6.0): Pending...
  • Installing typing-extensions (4.1.1): Pending...
  • Installing zipp (3.6.0): Pending...
  • Installing six (1.16.0): Downloading... 100%
  • Installing typing-extensions (4.1.1): Pending...
  • Installing zipp (3.6.0): Pending...
  • Installing typing-extensions (4.1.1): Pending...
  • Installing zipp (3.6.0): Pending...
  • Installing six (1.16.0): 

In [7]:
%cd src/
!poetry run pip3 install -e .

/content/evaluating-deeplight-transfer/src
Obtaining file:///content/evaluating-deeplight-transfer/src
  Preparing metadata (setup.py) ... done
  Running setup.py develop for deeplight


In [8]:
!mkdir ~/.aws

In [9]:
%%writefile ~/.aws/config

[profile hcp]
region=eu-central-1

Writing /root/.aws/config


In [10]:
%cd ../scripts/

/content/evaluating-deeplight-transfer/scripts


In [11]:
# edit download to keep task "WM"

In [12]:
%%writefile /content/evaluating-deeplight-transfer/scripts/download.py

#!/usr/bin/python
import os
import argparse
import numpy as np
import hcprep


def main():

  np.random.seed(13089)

  ap = argparse.ArgumentParser()
  ap.add_argument(
    "--ACCESS-KEY",
    required=True,
    help="AWS S3 access key"
  )
  ap.add_argument(
    "--SECRET-KEY",
    required=True,
    help="AWS S3 secret key"
  )
  ap.add_argument(
    "--path",
    required=False,
    default='../data/',
    help="path to store data (default: ../data/)"
  )
  ap.add_argument(
    "-n",
    required=False,
    default=3,
    help="number of subjects to download per HCP task (1-500) (default: 3)"
  )
  
  args = ap.parse_args()
  ACCESS_KEY = str(args.ACCESS_KEY)
  SECRET_KEY = str(args.SECRET_KEY)
  n = int(args.n)
  path = str(args.path)
  hcprep.paths.make_sure_path_exists(path)

  hcp_info = hcprep.info.basics()    

  print(
    'Downloading data of {} subjects to {}'.format(
      n, path
    )
  )
  #for task in hcp_info.tasks:
  task='WM'
  for subject in hcp_info.subjects[task][:n]:
      for run in hcp_info.runs:
        hcprep.download.download_subject_data(
          ACCESS_KEY=ACCESS_KEY,
          SECRET_KEY=SECRET_KEY,
          subject=subject,
          task=task,
          run=run,
          output_path=path
        )


if __name__ == '__main__':
  
  main()

Overwriting /content/evaluating-deeplight-transfer/scripts/download.py


In [13]:
!poetry run python download.py -n 8

downloading HCP/100307/MNINonLinear/Results/tfMRI_WM_LR/tfMRI_WM_LR.nii.gz  to  ../data/sub-100307/func/sub-100307_task-WM_run-LR_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz
downloading HCP/100307/MNINonLinear/Results/tfMRI_WM_LR/brainmask_fs.2.nii.gz  to  ../data/sub-100307/func/sub-100307_task-WM_run-LR_space-MNI152NLin6Asym_res-2_desc-preproc_brain_mask.nii.gz
downloading HCP/100307/MNINonLinear/T1w.nii.gz  to  ../data/sub-100307/anat/sub-100307_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz
downloading HCP/100307/MNINonLinear/Results/tfMRI_WM_LR/EVs/0bk_body.txt  to  ../data/sub-100307/func/sub-100307_task-WM_run-LR_desc-EV_0bk_body.txt
downloading HCP/100307/MNINonLinear/Results/tfMRI_WM_LR/EVs/0bk_faces.txt  to  ../data/sub-100307/func/sub-100307_task-WM_run-LR_desc-EV_0bk_faces.txt
downloading HCP/100307/MNINonLinear/Results/tfMRI_WM_LR/EVs/0bk_places.txt  to  ../data/sub-100307/func/sub-100307_task-WM_run-LR_desc-EV_0bk_places.txt
downloading HCP/100307/MNINonLine

In [16]:
!du -sh ../data

7.7G	../data


In [ ]:
!poetry run python preprocess.py

Using TensorFlow backend.
Processing 8 subjects with 2 runs per task.

Processing subject: 1/8
Skipping subject 101107 task EMOTION run LR, because BIDS data not fully present.
Skipping subject 101107 task EMOTION run RL, because BIDS data not fully present.
Skipping subject 101107 task GAMBLING run LR, because BIDS data not fully present.
Skipping subject 101107 task GAMBLING run RL, because BIDS data not fully present.
Skipping subject 101107 task LANGUAGE run LR, because BIDS data not fully present.
Skipping subject 101107 task LANGUAGE run RL, because BIDS data not fully present.
Skipping subject 101107 task MOTOR run LR, because BIDS data not fully present.
Skipping subject 101107 task MOTOR run RL, because BIDS data not fully present.
Skipping subject 101107 task RELATIONAL run LR, because BIDS data not fully present.
Skipping subject 101107 task RELATIONAL run RL, because BIDS data not fully present.
Skipping subject 101107 task SOCIAL run LR, because BIDS data not fully present

In [ ]:
!poetry run python train.py --batch-size=2